In [45]:
import torch # 파이토치 기본 라이브러리 
import torchvision # 이미지 관련 된 파이토치 라이브러리
from torchvision import datasets # 토치비전에서 제공하는 데이터셋
from torchvision import transforms # 이미지 전처리 기능들을 제공하는 라이브러리
from torch.utils.data import DataLoader # 데이터를 모델에 사용할 수 있도록 적재해 주는 라이브러리
from torch.utils.data import random_split
import numpy as np 
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.layers import Input, Conv2D, Dropout, Flatten, Activation, MaxPooling2D, Dense
from tensorflow.keras.layers import GlobalAveragePooling2D, BatchNormalization

from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Add, ZeroPadding2D
from tensorflow.keras.callbacks import TensorBoard

In [38]:
np.random.seed(42)
tf.random.set_seed(42)

# 데이터 불러오기

In [39]:
data_dir = "../input/chest-xray-pneumonia/chest_xray/"

In [49]:
transform =transforms.Compose([transforms.Resize([224, 224]), transforms.ToTensor()])

In [50]:
trainset = datasets.ImageFolder(root= data_dir + "/train/", transform=transform)
valset = datasets.ImageFolder(root= data_dir + "/val/", transform=transform)
testset = datasets.ImageFolder(root= data_dir + "/test/", transform=transform)

In [51]:
print(type(trainset), type(testset), type(valset))
print(len(trainset), len(testset), len(valset))

# 데이터 적재

In [43]:
batch_size = 16 # 중요한 하이퍼 파라미터, 16 이하로 사용하는것이 성능에 좋다고 알려져 있음
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True) # 훈련용
validloader = DataLoader(valset, batch_size=batch_size, shuffle=False) # 검증용
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False) # 테스트용

In [44]:
print(type(trainloader), len(trainloader))
print(type(validloader), len(validloader))
print(type(testloader), len(testloader))

In [45]:
dataiter = iter(trainloader)
images, labels = dataiter.next() # 16개의 image, 16개의 label
print(images.shape, labels.shape)

In [55]:
def imshow(img):         
    plt.figure(figsize=(20, 200))
    plt.imshow(img.permute(1, 2, 0))

dataiter = iter(trainloader)
images, labels = dataiter.next() # 32개의 image, 32개의 label

imshow(torchvision.utils.make_grid(images, nrows=8))

# 모델 생성

In [32]:
data_dir = "../input/chest-xray-pneumonia"

In [34]:
train_normal =glob.glob(data_dir + '/chest_xray/train/NORMAL/*.jpeg')
train_pneumonia =glob.glob(data_dir + '/chest_xray/train/PNEUMONIA/*.jpeg')

val_normal =glob.glob(data_dir + '/chest_xray/val/NORMAL/*.jpeg')
val_pneumonia =glob.glob(data_dir + '/chest_xray/val/PNEUMONIA/*.jpeg')

test_normal =glob.glob(data_dir + '/chest_xray/test/NORMAL/*.jpeg')
test_pneumonia =glob.glob(data_dir + '/chest_xray/test/PNEUMONIA/*.jpeg')

In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split

# train_df
train_path = train_normal + train_pneumonia
train_label = ["NORMAL" for _ in range(len(train_normal))] + ["PNEUMONIA" for _ in range(len(train_pneumonia))]
train_df = pd.DataFrame({"path":train_path, "label":train_label})

# test_df
test_paths = test_normal + test_pneumonia
test_label = ["NORMAL" for _ in range(len(test_normal))] + ["PNEUMONIA" for _ in range(len(test_pneumonia))]
test_df = pd.DataFrame({"path":test_paths, "label":test_label})

# val_df (조정해야할거같은느낌)
train_df, val_df = train_test_split(train_df, test_size=0.15, stratify=train_df['label'], random_state=42)

print(train_df['label'].value_counts())
print(test_df['label'].value_counts())
print(val_df['label'].value_counts())

In [36]:
pd.set_option("display.max_colwidth", 200)
train_df.head()

In [46]:
BATCH_SIZE = 16
# train_generator
train_generator = ImageDataGenerator(horizontal_flip= True,
                                     rotation_range = 0.1,
                                     zoom_range = 0.2,
                                     fill_mode='nearest',# 외부영역 
                                     width_shift_range=0.1,
                                     height_shift_range=0.1,
                                     rescale = 1/255.0)

train_generator_iterator = train_generator.flow_from_dataframe(dataframe=train_df,
                                                               x_col = 'path',
                                                               y_col = 'label',
                                                               target_size = (224, 224), class_mode = "binary",
                                                               batch_size = BATCH_SIZE, shuffle=True)

# valid_generator
valid_generator = ImageDataGenerator(rescale = 1/255.0)

valid_generator_iterator = valid_generator.flow_from_dataframe(dataframe=val_df,
                                                               x_col = 'path',
                                                               y_col = 'label',
                                                               target_size = (224, 224), class_mode = "binary",
                                                               batch_size = 8, shuffle=False)


# test_generator
test_generator = ImageDataGenerator(rescale = 1/255.0)

test_generator_iterator = test_generator.flow_from_dataframe(dataframe=test_df,
                                                               x_col = 'path',
                                                               y_col = 'label',
                                                               target_size = (224, 224), class_mode = "binary",
                                                               batch_size = BATCH_SIZE, shuffle=False)

In [47]:
train_batch, label_batch = next(train_generator_iterator)
train_batch.shape, label_batch.shape

In [48]:
# 언제 어떻게 활용될지 모르니까 다 import(케라스사용)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.layers import Input, Conv2D, Dropout, Flatten, Activation, MaxPooling2D, Dense
from tensorflow.keras.layers import GlobalAveragePooling2D, BatchNormalization

from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Add, ZeroPadding2D
from tensorflow.keras.callbacks import TensorBoard

In [49]:
def identity_block(input_tensor, filters):
    filter1, filter2, filter3 = filters
    
    x = Conv2D(filters=filter1, kernel_size=(1, 1), padding="same", kernel_initializer="he_normal")(input_tensor)
    x = BatchNormalization(axis=3)(x)
    x = Activation("relu")(x)
    
    x = Conv2D(filters=filter2, kernel_size=(3, 3), padding="same", kernel_initializer="he_normal")(x)
    x = BatchNormalization(axis=3)(x)
    x = Activation("relu")(x)
    
    x = Conv2D(filters=filter3, kernel_size=(1, 1), padding="same", kernel_initializer="he_normal")(x)
    x = BatchNormalization(axis=3)(x)

    x = Add()([input_tensor, x])
    output = Activation("relu")(x)
    
    return output

In [50]:
def convolutional_block(input_tensor, filters, strides=(2,2)):
    filter1, filter2, filter3 = filters
    
    x = Conv2D(filters=filter1, kernel_size=(1, 1), padding="same", strides = strides, kernel_initializer="he_normal")(input_tensor)
    x = BatchNormalization(axis=3)(x)
    x = Activation("relu")(x)
    
    x = Conv2D(filters=filter2, kernel_size=(3, 3), padding="same", kernel_initializer="he_normal")(x)
    x = BatchNormalization(axis=3)(x)
    x = Activation("relu")(x)
    
    x = Conv2D(filters=filter3, kernel_size=(1, 1), padding="same", kernel_initializer="he_normal")(x)
    x = BatchNormalization(axis=3)(x)

    shortcut = Conv2D(filters=filter3, kernel_size=(1, 1), padding="same", strides=strides, kernel_initializer="he_normal")(input_tensor)
    shortcut = BatchNormalization(axis=3)(shortcut)
    
    x = Add()([x, shortcut])
    output = Activation("relu")(x)
    
    return output

In [51]:
def build_resnet50():
    tf.keras.backend.clear_session()
    input_tensor = Input(shape=(224, 224, 3))

    x = ZeroPadding2D(padding=(3,3))(input_tensor)
    x = Conv2D(filters=64, kernel_size=(7,7), padding="valid", strides=(2,2))(x)
    x = BatchNormalization(axis=3)(x)
    x = Activation("relu")(x)

    x = ZeroPadding2D(padding=(1,1))(x)
    x = MaxPooling2D(pool_size=(3,3), strides=(2,2))(x)

    x = convolutional_block(x, [64, 64, 256], strides=(1, 1))
    x = identity_block(x, [64, 64, 256])
    x = identity_block(x, [64, 64, 256])

    x = convolutional_block(x, [128, 128, 512], strides=(2, 2))
    x = identity_block(x, [128, 128, 512])
    x = identity_block(x, [128, 128, 512])
    x = identity_block(x, [128, 128, 512])

    x = convolutional_block(x, [256, 256, 1024], strides=(2, 2))
    x = identity_block(x, [256, 256, 1024])
    x = identity_block(x, [256, 256, 1024])
    x = identity_block(x, [256, 256, 1024])
    x = identity_block(x, [256, 256, 1024])
    x = identity_block(x, [256, 256, 1024])

    x = convolutional_block(x, [512, 512, 2048], strides=(2, 2))
    x = identity_block(x, [512, 512, 2048])
    x = identity_block(x, [512, 512, 2048])

    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(200, activation="relu")(x)
    x = Dropout(0.5)(x)
    output = Dense(1, activation="sigmoid")(x)

    model = Model(inputs=input_tensor, outputs= output)
    #model.compile(optimizer=Adam(0.0001), loss="binary_crossentropy", metrics=["accuracy"])
    model.compile(optimizer=SGD(0.01, momentum=0.9), loss="binary_crossentropy", metrics=["accuracy"])
    
    return model
model = build_resnet50()
model.summary()

In [52]:
import os
import time
root_logdir = os.path.join(os.curdir, "my_logs")

def get_run_logdir():
  run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
  return os.path.join(root_logdir, run_id)

In [53]:
run_logdir = get_run_logdir()
checkpoint_cb = ModelCheckpoint("my_best_model.h5", monitor="val_loss", save_best_only = True, verbose=1)
early_stopping_cb = EarlyStopping( monitor="val_loss", patience=10, verbose=1, restore_best_weights=False)
reduce_lr_cb = ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, verbose=1)
tensorboard_cb = TensorBoard(run_logdir)

In [54]:
history = model.fit(train_generator_iterator, validation_data=valid_generator_iterator, epochs=30, 
                    callbacks=[checkpoint_cb, early_stopping_cb, reduce_lr_cb, tensorboard_cb])

# 모델 평가

In [55]:
#using model.predict as per keras rule to use >0.5 for binary classification
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
pred = (model.predict(test_generator_iterator)>0.5).astype("int32")

y_test = test_generator_iterator.labels
print('Classification report:\n', classification_report(y_test, pred))
print('Accuracy score:\n', accuracy_score(y_test, pred))
pd.DataFrame(classification_report(test_generator_iterator.classes, pred > 0.5, output_dict=True))

In [50]:
model.evaluate(test_generator_iterator)

In [25]:
plt.plot(history.history['loss'], label="train loss")
plt.plot(history.history['val_loss'], label="validation loss")
plt.legend()

In [51]:
plt.plot(history.history['accuracy'], label="train accuracy")
plt.plot(history.history['val_accuracy'], label="validation accuracy")
plt.legend()